# TAQ midpoint price data

En la implementacion del midpoint price, divido la tarea en dos partes. Primero obtengo los resultados para todos los eventos en un dia y luego con otra funcion calculo todos los dias del ano.

### midpoint price data por evento por dia

Esta funcion no tiene problema en cuanto al tiempo de ejecucion. Las operaciones que realiza son muy basicas. Sin embargo, mi implementacion anterior usando solo `numpy` se demoraba 0,020s por dia y ahora se demora 0.1s por dia utilizando `pandas`.

La implementacion con `numpy` la voy a nombrar `taq_midpoint_event_data_numpy` y la implementacion con `pandas` la voy a nombrar `taq_midpoint_event_data_pandas`.

En este caso no entiendo por que con `pandas` es mas lento

In [1]:
# Modules

import numpy as np
import pandas as pd
import pickle

In [2]:
def taq_midpoint_event_data_numpy(ticker, year, month, day):
    """Computes the midpoint price of every event.

    Using the dayly TAQ data computes the midpoint price of every event in a
    day.
    For further calculations, the function returns the values for the time
    range from 9h40 to 15h50.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param year: string of the year to be analized (i.e '2008').
    :param month: string of the month to be analized (i.e '07').
    :param day: string of the day to be analized (i.e '07').
    :return: tuple -- The function returns a tuple with numpy arrays.
    """

    # Load data
    # TAQ data gives directly the quotes data in every second that there is
    # a change in the quotes
    time_q_, bid_q_, ask_q_, _, _ = pickle.load(open(
        f'pickle_dayly_data_{year}/TAQ_{ticker}_quotes_{year}{month}{day}.pickle', 'rb'))

    # Some files are corrupted, so there are some zero values that
    # does not have sense
    condition_1 = ask_q_ != 0.
    time_q = time_q_[condition_1]
    bid_q = bid_q_[condition_1]
    ask_q = ask_q_[condition_1]

    assert len(bid_q) == len(ask_q)

    midpoint = (bid_q + ask_q) / 2
    spread = ask_q - bid_q

    return (time_q, bid_q, ask_q, midpoint, spread)

In [3]:
def taq_midpoint_event_data_pandas(ticker, date):
    """Computes the midpoint price of every event.

    Using the dayly TAQ data computes the midpoint price of every event in a
    day.
    For further calculations, the function returns the values for the time
    range from 9h40 to 15h50.

    :param ticker: string of the abbreviation of the stock to be analized
     (i.e. 'AAPL').
    :param year: string of the year to be analized (i.e '2008').
    :param month: string of the month to be analized (i.e '07').
    :param day: string of the day to be analized (i.e '07').
    :return: tuple -- The function returns a tuple with numpy arrays.
    """

    date_sep = date.split('-')
    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    # Load data
    # TAQ data gives directly the quotes data in every second that there is
    # a change in the quotes
    data_quotes_event = pd.read_hdf(
        f'hdf5_dayly_data_{year}/taq_{ticker}_quotes_{date}.h5')

    # Some files are corrupted, so there are some zero values that
    # does not have sense
    data_quotes_event = data_quotes_event[data_quotes_event['Ask'] != 0]

    data_quotes_event['Midpoint'] = (data_quotes_event['Bid']
                                     + data_quotes_event['Ask']) / 2
    data_quotes_event['Spread'] = data_quotes_event['Ask'] \
        - data_quotes_event['Bid']

    return data_quotes_event

In [4]:
ticker = 'AAPL'
year = '2008'
month = '01'
day = '02'
date = f'{year}-{month}-{day}'

In [8]:
%%timeit
taq_midpoint_event_data_numpy(ticker, year, month, day)

9.25 ms ± 81.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
taq_midpoint_event_data_pandas(ticker, date)

36.9 ms ± 522 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### midpoint price data por segundo por dia